In [ ]:
# from pyspark.sql import functions

In [1]:
from py4j.protocol import Py4JJavaError
from pyspark import SparkConf, SparkContext
import pyspark
from pyspark.sql import functions,SparkSession, SQLContext
from pyspark.sql.functions import explode, col, udf, concat_ws, from_json, lit, array, expr, size
from pyspark.sql.functions import sum as _sum
from pyspark.sql.types import *
import json
import os
import gc
from pyspark.sql.types import BooleanType
# from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1681467224689_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
input_bucket = 's3://cs5344-twitter-project'
path = '/{}/*.json.gz'
folder_list = ['20220122', '20220123','20220124','20220125', 
               '20220126', '20220127','20220128', '20220129',
               '20220130', '20220131', '20220201', '20220202',
               '20220203', '20220204', '20220205', '20220206',
               '20220207', '20220208', '20220209', '20220210',
               '20220211', '20220212', '20220213', '20220214',
               '20220215']


input_bucket = 's3://cs5344-twitter-project'
test_pokemon_damping_sdf = spark.read.format('csv').options(header='False').load('s3://cs5344-twitter-project/test_pokemon_damping').cache()
pokemon = test_pokemon_damping_sdf.select(col("_c0").alias("user_id_str")).cache()
pokemon.createOrReplaceTempView("pokemon")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
#  "user.verified",
#         "user.followers_count",
#         "user.friends_count",
#         "user.favourites_count",
#         "user.statuses_count",
#         "user.created_at",

In [3]:
def pre_process(folder):
    raw = spark.read.json(folder, allowBackslashEscapingAnyCharacter=True)

    user_df = raw.select(
        col("user.id_str").alias("user_id_str"),
        col("user.screen_name").alias("user_twitter_handle"),
        "id_str",
        col("created_at").alias("tweet_created_at"),
        "quote_count",
        "reply_count",
        "retweet_count",
        col("favorite_count").alias("tweet_favorite_count")
    )
    user_df.cache()
    user_df.createOrReplaceTempView("user_df")
    
#     ier prepare
    tweet_interact =  spark.sql("""
    select a.user_id_str,
            a.id_str,
           a.interact_cnt
    from 
    (select  user_id_str,
            id_str,
           (max_quotes_count +  max_replies_count +  max_retweets_count) as interact_cnt
    from 
        (select user_id_str,
               id_str,
            max(quote_count) max_quotes_count,
            max(reply_count) max_replies_count,
            max(retweet_count) max_retweets_count
        from user_df
        group by user_id_str,id_str) 
    where (max_quotes_count + max_replies_count +  max_retweets_count) > 0) a
    join pokemon b
    on a.user_id_str = b.user_id_str
    """)
    # latest info of influ    
#     user_join = spark.sql("""
#     select user_filter.*
#     from
#         (select *
#         from
#             (select user_id_str, 
#                     user_twitter_handle, 
#                     verified, 
#                     followers_count, 
#                     friends_count, 
#                     favourites_count, 
#                     statuses_count, 
#                     created_at,
#                     tweet_created_at,
#                     rank() over(partition by user_id_str order by tweet_created_at desc) create_rank
#             from user_df
#             where user_id_str is not null) u
#         where create_rank = 1) user_filter
#     join diff
#     on user_filter.user_id_str = diff.user_id_str
#     """)
#     user_filter.createOrReplaceTempView("user_filter")

#     user_join = spark.sql("""
#     select user_filter.*
#     from
#     user_filter
#     join
#     diff
#     on user_filter.user_id_str = diff.user_id_str""")
#     return user_join, tweet_interact
#     return user_join
    return tweet_interact



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
f = folder_list[2]
folder = input_bucket + path.format(f)

raw = spark.read.json(folder, allowBackslashEscapingAnyCharacter=True)

user_df = raw.select(
    col("user.id_str").alias("user_id_str"),
    col("user.screen_name").alias("user_twitter_handle"),
    "id_str",
    col("created_at").alias("tweet_created_at"),
    "quote_count",
    "reply_count",
    "retweet_count",
    col("favorite_count").alias("tweet_favorite_count")
)
# user_df.cache()
user_df.createOrReplaceTempView("user_df")
tmp =  spark.sql("""
select max(quote_count),
    max(reply_count),
    max(retweet_count)
    from user_df""")
tmp.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+----------------+------------------+
|max(quote_count)|max(reply_count)|max(retweet_count)|
+----------------+----------------+------------------+
|               0|               0|                 0|
+----------------+----------------+------------------+

In [22]:
# user_df
tweet_interact =  spark.sql("""
select user_id_str,id_str, max(quote_count),max(reply_count),max(retweet_count) from user_df group by user_id_str, id_str""")
tweet_interact.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+-------------------+----------------+----------------+------------------+
|        user_id_str|             id_str|max(quote_count)|max(reply_count)|max(retweet_count)|
+-------------------+-------------------+----------------+----------------+------------------+
|1346314039459258368|1484904307099058176|               0|               0|                 0|
|1464749508785545218|1484904361646071814|               0|               0|                 0|
|         4882676778|1484904017725583362|               0|               0|                 0|
|1228262977557516289|1484904055474647040|               0|               0|                 0|
|1364981724480856065|1484904076425175045|               0|               0|                 0|
|1348492234841149442|1484904080606523392|               0|               0|                 0|
|1249338947169214464|1484904147744788482|               0|               0|                 0|
| 957873352596602881|1484904487479169030|         

In [9]:
diff = pokemon.alias("diff")
tweet_interact_info = None
pokemon_inf = None
for f in folder_list:
    folder = input_bucket + path.format(f)
    try:
#         user_join, tweet_interact = pre_process(folder, diff)
#         user_join = pre_process(folder, diff)
        tweet_interact = pre_process(folder, diff)

    #         if diff.count() == 0:
    #             print('--------------------all influ have been found--------------------')
    #             break
#         diff = diff.join(user_join, on="user_id_str", how="left_anti")
#         if pokemon_inf:
#             pokemon_inf = pokemon_inf.union(user_join)
#         else:
#             pokemon_inf = user_join.alias("pokemon_inf")

        if tweet_interact_info:
            tweet_interact_info =tweet_interact_info.union(tweet_interact)
        else:
            tweet_interact_info =tweet_interact.alias("tweet_interact")
#         del user_join
        del tweet_interact
        gc.collect()
    except Py4JJavaError:
        print(folder+" failed")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

668
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204

In [ ]:
tweet_interact_info.show(50)

In [11]:
output_bucket = 's3://cs5344-twitter-project/'
# pokemon_inf.toDF().write.mode('overwrite').csv(output_bucket+'pokemon_inf_24_aws') 
tweet_interact_info.write.format("csv").save(output_bucket+'user_ana/tweet_interact') 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
tweet_interact_info.createOrReplaceTempView("tweet_interact_info")
ier_0 = spark.sql("""
select user_id_str, sum(interact_cnt) from(select user_id_str, id_str, max(interact_cnt)
from tweet_interact_info
group by user_id_str, id_str)
group by user_id_str""")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
ier_0.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+-----------------+
|user_id_str|sum(interact_cnt)|
+-----------+-----------------+
+-----------+-----------------+

In [ ]:
# tweet_interact_info.show()

In [ ]:
diff = pokemon.alias("diff")
pokemon_inf = None
for f in folder_list[-6:-16:-1]:
    folder = input_bucket + path.format(f)
    try:
        user_join = pre_process(folder)
#         diff = diff.join(user_join, on="user_id_str", how="left_anti")
        pokemon_inf = user_join.alias("pokemon_inf")
        del user_join
        gc.collect()
    except Py4JJavaError:
        print(folder+" failed")

In [ ]:
pokemon_inf.show(50, truncate= False)

In [ ]:
# diff = pokemon.alias("diff")
pokemon_inf = None
for f in folder_list[-16:-26:-1]:
    folder = input_bucket + path.format(f)
    try:
        user_join_2 = pre_process(folder)
#         diff = diff.join(user_join, on="user_id_str", how="left_anti")
#         pokemon_inf = user_join.alias("pokemon_inf")
#         del user_join
        gc.collect()
    except Py4JJavaError:
        print(folder+" failed")

In [ ]:
user_join_2.show(50, truncate= False)

In [ ]:
folder_list[-16:-26:-1]

In [ ]:
for f in folder_list[-11:-16:-1]:
    folder = input_bucket + path.format(f)
    try:
        user_join, diff = pre_process(folder,diff)
        if diff.count() == 0:
            print('--------------------all influ have been found--------------------')
            break
        diff = diff.join(user_join, on="user_id_str", how="left_anti")
        if pokemon_inf:
            pokemon_inf = pokemon_inf.union(user_join)
        else:
            pokemon_inf = user_join.alias("pokemon_inf")
        del user_join
        gc.collect()
    except Py4JJavaError:
        print(folder+" failed")

In [ ]:
pokemon_inf.show()

In [ ]:
for f in folder_list[-16:-21:-1]:
    folder = input_bucket + path.format(f)
    try:
        user_join, diff = pre_process(folder,diff)
        if diff.count() == 0:
            print('--------------------all influ have been found--------------------')
            break
        diff = diff.join(user_join, on="user_id_str", how="left_anti")
        if pokemon_inf:
            pokemon_inf = pokemon_inf.union(user_join)
        else:
            pokemon_inf = user_join.alias("pokemon_inf")
        del user_join
        gc.collect()
    except Py4JJavaError:
        print(folder+" failed")

In [ ]:
pokemon_inf.show()

In [ ]:
for f in folder_list[-21:-26:-1]:
    folder = input_bucket + path.format(f)
    try:
        user_join, diff = pre_process(folder,diff)
        if diff.count() == 0:
            print('--------------------all influ have been found--------------------')
            break
        diff = diff.join(user_join, on="user_id_str", how="left_anti")
        if pokemon_inf:
            pokemon_inf = pokemon_inf.union(user_join)
        else:
            pokemon_inf = user_join.alias("pokemon_inf")
        del user_join
        gc.collect()
    except Py4JJavaError:
        print(folder+" failed")

In [ ]:
pokemon_inf.show()

In [ ]:
pokemon_inf.count()

In [ ]:
diff.count()

In [ ]:
diff.show()

In [ ]:
user_join.show()

In [ ]:
pokemon_inf.show()

In [ ]:
# IER = (retweets + favorites) / followers

In [ ]:
# raw.select().show()

In [ ]:
from pyspark.sql.functions import sum, first

# Filter DataFrame to include only tweets by a particular influencer
influencer_tweets = tweets_df.filter(tweets_df.user_id == influencer_id)

# Calculate total number of retweets and favorites for the influencer's tweets
total_retweets = influencer_tweets.agg(sum("retweets")).first()[0]
total_favorites = influencer_tweets.agg(sum("favorites")).first()[0]

# Get the number of followers for the influencer
follower_count = influencer_tweets.select("followers").distinct().first()[0]

# Calculate the IER
ier = (total_retweets + total_favorites) / follower_count

In [ ]:
diff.count(), pokemon_inf.count()

In [ ]:
pokemon.count() == 0

In [ ]:
pokemon_inf.show()

In [ ]:
res.show()

In [ ]:
res.count()

In [ ]:
print("-"*20 + "user_filter" + "-"*20)
